# Multiturn injection analysis

In [ ]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import proplot as pplt
import psdist as ps
import psdist.visualization as psv
import yaml
from ipywidgets import interact
from ipywidgets import widgets
from omegaconf import OmegaConf
from omegaconf import DictConfig
from pprint import pprint

In [ ]:
pplt.rc["cmap.discrete"] = False
pplt.rc["cmap.sequential"] = "viridis"
pplt.rc["cycle"] = "538"
pplt.rc["grid"] = False
pplt.rc["figure.facecolor"] = "white"

## Setup

In [ ]:
timestamp = None
script_name = "paint_root"

if timestamp is None:
    input_dirs = os.listdir(f"./outputs/{script_name}")
    input_dirs = sorted(input_dirs)
    input_dir = input_dirs[-1]
    input_dir = os.path.join(f"./outputs/{script_name}", input_dir)
else:
    input_dir = f"./outputs/{script_name}/{timestamp}/"

print("intput_dir = ", input_dir)

In [ ]:
cfg_path = os.path.join(input_dir, "config/config.yaml")
cfg = yaml.safe_load(open(cfg_path, "r"))
cfg = DictConfig(cfg)

print("config:")
print(OmegaConf.to_yaml(cfg))

In [ ]:
cycle_colors = pplt.Cycle(pplt.rc["cycle"]).by_key()["color"]

## Scalars

In [ ]:
history = pd.read_csv(os.path.join(input_dir, "history.dat"))
history.head()

In [ ]:
pprint(list(history.keys()))

In [ ]:
fig, ax = pplt.subplots(figsize=(4.5, 2.0))
for key in ["x_rms", "y_rms"]:
    ax.plot(history[key].values * 1000.0, label=key)
ax.legend(loc="r", ncols=1)
ax.format(xlabel="Turn", ylabel="[mm]", ymin=0.0)
ax.format(xmin=-20, xmax=(history.shape[0] + 1))

In [ ]:
fig, ax = pplt.subplots(figsize=(4.5, 2.0))
for key in ["eps_x", "eps_y"]:
    ax.plot(history[key].values * 1.00e+06, label=key)
ax.legend(loc="r", ncols=1)
ax.format(xlabel="Turn", ylabel="[mm mrad]", ymin=0.0)
ax.format(xmin=-20, xmax=(history.shape[0] + 1))

In [ ]:
fig, ax = pplt.subplots(figsize=(4.5, 2.0))
for i, key in enumerate(["eps_x", "eps_y"]):
    ax.plot(history[key].values * 1.00e+06, label=key, color=cycle_colors[i])
for i, key in enumerate(["eps_1", "eps_2"]):
    ax.plot(history[key].values * 1.00e+06, label=key, color=cycle_colors[i], ls=":")
ax.legend(loc="r", ncols=1)
ax.format(xlabel="Turn", ylabel="[mm mrad]", ymin=0.0)
ax.format(xmin=-20, xmax=(history.shape[0] + 1))

## Tunes

In [ ]:
# [...]

## Phase space distribution

In [ ]:
bunch_filenames = os.listdir(input_dir)
bunch_filenames = sorted(bunch_filenames)
bunch_filenames = [f for f in bunch_filenames if f.startswith("bunch")]
bunch_filenames = [os.path.join(input_dir, f) for f in bunch_filenames]

bunches = []
bunch_turns = []
for filename in bunch_filenames:
    X = np.loadtxt(filename, comments="%", usecols=range(6))
    X[:, :4] *= 1000.0
    X[:, 5] *= 1000.0
    X = X - np.mean(X, axis=0)
    bunches.append(X)

    turn = int(filename.split("_")[-1].split(".dat")[0])
    bunch_turns.append(turn)

In [ ]:
dims = ["x", "xp", "y", "yp", "z", "dE"]
units = ["mm", "mrad", "mm", "mrad", "m", "MeV"]
labels = [f"{dim} [{unit}]" for dim, unit in zip(dims, units)]
limits = ps.points.limits(bunches[-1], zero_center=True, share=[(0, 2), (1, 3)], pad=0.25)

### Interactive 2D projections

In [ ]:
@interact(
    dim1=widgets.Dropdown(options=dims, value=dims[0]),
    dim2=widgets.Dropdown(options=dims, value=dims[1]),
    index=widgets.IntSlider(min=0, max=(len(bunches) - 1), value=0),
    bins=widgets.IntSlider(min=32, max=128, value=64),
)
def update(dim1: str, dim2: str, index: int, bins: int):
    if dim1 == dim2:
        return

    axis = [dims.index(dim) for dim in [dim1, dim2]]
    axis = tuple(axis)
    
    X = bunches[index]

    fig, ax = pplt.subplots()
    ax.hist2d(
        X[:, axis[0]], 
        X[:, axis[1]], 
        bins=bins,
        range=[limits[axis[0]], limits[axis[1]]],
    )
    ax.format(xlabel=labels[axis[0]], ylabel=labels[axis[1]])
    ax.format(title=f"turn = {bunch_turns[index]}")

    paxs = [ax.panel_axes(loc) for loc in ["top", "right"]]
    for pax in paxs:
        pax.format(xspineloc="bottom", yspineloc="left")
    kws = dict(bins=bins, density=False, color="black", histtype="step", lw=1.25)
    paxs[0].hist( X[:, axis[0]], range=limits[axis[0]], **kws)
    paxs[1].histh(X[:, axis[1]], range=limits[axis[1]], **kws)
    plt.show()

### Interactive corner

In [ ]:
@interact(
    ndim=widgets.BoundedIntText(min=4, max=6, value=4),
    index=widgets.IntSlider(min=0, max=(len(bunches) - 1), value=0),
    bins=widgets.IntSlider(min=32, max=128, value=42),
    ellipse=False,
)
def update(ndim: int, index: int, bins: int, ellipse: bool):    
    X = bunches[index][:, :ndim]
    
    grid = psv.CornerGrid(ndim, diag_shrink=0.85)
    grid.plot_points(
        X,
        bins=bins,
        limits=limits,
        mask=False,
        rms_ellipse=ellipse,
        rms_ellipse_kws=dict(level=2.0, color="white"),
    )
    grid.set_labels(labels)
    grid.axs.format(suptitle=f"Turn = {bunch_turns[index]}")
    plt.show()